# AlphaFold Model Variation Analysis

This notebook analyzes **structural variation between different AlphaFold prediction models** (model_0, model_1, model_2, etc.) for PKS proteins.

## Purpose
- Quantify how much structural predictions vary between models
- Identify which structural features are most/least variable
- Understand implications for using single vs. ensemble predictions
- Guide decisions about which model(s) to use for downstream analysis

## Key Questions
1. How consistent are structural measurements across models?
2. Do some domains show more variation than others?
3. Are certain properties (SASA, Rg, volume) more robust than others?
4. Should we average across models or use a single representative?


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 100)

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

print("Libraries loaded successfully")


---
## 1. Load Data and Extract Model Information


In [ ]:
# Load both domain and module macroproperties
domain_mp = pd.read_csv('domain_macroproperties.csv', index_col=0)
module_mp = pd.read_csv('MP_PKS.csv')

print(f"Domain macroproperties: {domain_mp.shape}")
print(f"Module macroproperties: {module_mp.shape}")


In [ ]:
# Extract model number and base structure from filenames
def extract_model_info(filename):
    """Extract model number and base structure identifier from filename"""
    # Pattern: domain_MP_fold_BGC0000001p1_abyB1_SingleModule_M0_Core_model_0_ACP_0860-0930.npz
    # or: MP_fold_bgc0000001p1_abyb1_singlemodule_m0_core_model_0.npz
    
    # Extract model number
    model_match = re.search(r'model_(\d+)', filename.lower())
    model_num = int(model_match.group(1)) if model_match else None
    
    # Extract base structure (everything before model_X)
    base_match = re.search(r'(.+)_model_\d+', filename.lower())
    base_structure = base_match.group(1) if base_match else None
    
    # Extract domain type (for domain data)
    domain_match = re.search(r'_([A-Za-z]+)_\d+-\d+\.npz$', filename)
    domain_type = domain_match.group(1) if domain_match else None
    
    # Extract BGC and module info
    bgc_match = re.search(r'bgc(\d+)p(\d+)', filename.lower())
    module_match = re.search(r'_(m\d+)_', filename.lower())
    
    return {
        'model_num': model_num,
        'base_structure': base_structure,
        'domain_type': domain_type,
        'bgc': bgc_match.group(0) if bgc_match else None,
        'module': module_match.group(1).upper() if module_match else None
    }

# Apply to domain data
domain_info = domain_mp['filename'].apply(extract_model_info).apply(pd.Series)
domain_mp = pd.concat([domain_mp, domain_info], axis=1)

# Apply to module data
module_info = module_mp['filename'].apply(extract_model_info).apply(pd.Series)
module_mp = pd.concat([module_mp, module_info], axis=1)

print("Model distribution in domain data:")
print(domain_mp['model_num'].value_counts().sort_index())
print(f"\nModel distribution in module data:")
print(module_mp['model_num'].value_counts().sort_index())


---
## 2. Module-Level Model Variation Analysis

Analyze how structural properties vary across AlphaFold models for the same module.


In [ ]:
# Calculate variation statistics for each unique module across its models
# Group by base_structure (which identifies the same protein/module)

numeric_cols = ['n_ca_atoms', 'n_heavy_atoms', 'radius_of_gyration_ca', 'sasa', 'ses_area', 'vdw_volume']

# Calculate per-structure variation (across models)
module_variation = module_mp.groupby('base_structure')[numeric_cols].agg(['mean', 'std', 'min', 'max', 'count'])

# Calculate coefficient of variation (CV = std/mean * 100)
module_cv = pd.DataFrame()
for col in numeric_cols:
    module_cv[col] = (module_variation[(col, 'std')] / module_variation[(col, 'mean')] * 100)

print("Coefficient of Variation (%) across models per structure:")
print(module_cv.describe().round(3))


In [ ]:
# Visualize coefficient of variation distributions
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

for i, col in enumerate(numeric_cols):
    ax = axes[i]
    data = module_cv[col].dropna()
    ax.hist(data, bins=50, edgecolor='black', alpha=0.7, color=plt.cm.viridis(i/len(numeric_cols)))
    ax.axvline(data.median(), color='red', linestyle='--', linewidth=2, label=f'Median: {data.median():.2f}%')
    ax.axvline(data.mean(), color='orange', linestyle=':', linewidth=2, label=f'Mean: {data.mean():.2f}%')
    ax.set_xlabel('Coefficient of Variation (%)')
    ax.set_ylabel('Count (structures)')
    ax.set_title(f'{col}\nModel-to-Model Variation')
    ax.legend(fontsize=9)

plt.suptitle('How Much Do AlphaFold Models Vary? (Module Level)', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

# Summary statistics
print("\n" + "="*60)
print("SUMMARY: Average Coefficient of Variation Across Models")
print("="*60)
for col in numeric_cols:
    cv = module_cv[col].dropna()
    print(f"{col:25s}: {cv.mean():.3f}% (median: {cv.median():.3f}%)")


In [ ]:
# Compare specific models (model_0 vs others)
# Create wide format for paired comparisons

model_comparison = module_mp.pivot_table(
    index='base_structure',
    columns='model_num',
    values=numeric_cols,
    aggfunc='first'
)

# Compare model 0 vs model 1, 2, 3, 4
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

for i, col in enumerate(numeric_cols):
    ax = axes[i]
    for model_num in [1, 2, 3, 4]:
        if (col, 0) in model_comparison.columns and (col, model_num) in model_comparison.columns:
            x = model_comparison[(col, 0)]
            y = model_comparison[(col, model_num)]
            mask = ~(x.isna() | y.isna())
            if mask.sum() > 10:
                corr = x[mask].corr(y[mask])
                ax.scatter(x[mask], y[mask], alpha=0.3, s=10, label=f'Model {model_num} (r={corr:.3f})')
    
    ax.plot([ax.get_xlim()[0], ax.get_xlim()[1]], 
            [ax.get_xlim()[0], ax.get_xlim()[1]], 'r--', linewidth=2, label='Perfect agreement')
    ax.set_xlabel(f'{col} (Model 0)')
    ax.set_ylabel(f'{col} (Other models)')
    ax.set_title(f'{col}')
    ax.legend(fontsize=8, loc='lower right')

plt.suptitle('Model 0 vs Other Models: Pairwise Comparisons', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()


---
## 3. Domain-Level Model Variation Analysis

Analyze how individual domain properties vary across models.


In [ ]:
# Create unique identifier for each domain instance across models
domain_mp['domain_instance'] = domain_mp['base_structure'] + '_' + domain_mp['domain_type'].astype(str)

# Calculate variation per domain instance
domain_variation = domain_mp.groupby('domain_instance')[numeric_cols].agg(['mean', 'std', 'count'])

# Calculate CV for domains
domain_cv = pd.DataFrame()
for col in numeric_cols:
    domain_cv[col] = (domain_variation[(col, 'std')] / domain_variation[(col, 'mean')] * 100)

# Add domain type back
domain_cv['domain_type'] = domain_cv.index.str.extract(r'_([A-Za-z]+)$')[0].values

print("Domain-level CV statistics:")
print(domain_cv[numeric_cols].describe().round(3))


In [ ]:
# Compare CV by domain type - which domains are most variable?
def is_linker(domain_type):
    if pd.isna(domain_type) or len(str(domain_type)) <= 2:
        return False
    return str(domain_type).endswith('L')

domain_cv['is_linker'] = domain_cv['domain_type'].apply(is_linker)

# Calculate mean CV by domain type for key property (radius of gyration)
cv_by_domain = domain_cv.groupby('domain_type')['radius_of_gyration_ca'].agg(['mean', 'std', 'count'])
cv_by_domain = cv_by_domain[cv_by_domain['count'] >= 10].sort_values('mean', ascending=False)

print("Model-to-Model Variation by Domain Type (Radius of Gyration):")
print(cv_by_domain.round(3).head(20))


In [ ]:
# Visualize domain variation by type
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. CV by domain type - Radius of gyration
ax = axes[0, 0]
colors = ['#e74c3c' if is_linker(d) else '#3498db' for d in cv_by_domain.index]
ax.barh(cv_by_domain.index, cv_by_domain['mean'], xerr=cv_by_domain['std'], 
        color=colors, alpha=0.7, capsize=3)
ax.set_xlabel('Mean CV (%) for Radius of Gyration')
ax.set_ylabel('Domain Type')
ax.set_title('Model Variation by Domain Type\n(Radius of Gyration)')
ax.invert_yaxis()

# 2. CV by domain type - SASA
cv_sasa = domain_cv.groupby('domain_type')['sasa'].agg(['mean', 'std', 'count'])
cv_sasa = cv_sasa[cv_sasa['count'] >= 10].sort_values('mean', ascending=False)
ax = axes[0, 1]
colors = ['#e74c3c' if is_linker(d) else '#3498db' for d in cv_sasa.index]
ax.barh(cv_sasa.index, cv_sasa['mean'], xerr=cv_sasa['std'], 
        color=colors, alpha=0.7, capsize=3)
ax.set_xlabel('Mean CV (%) for SASA')
ax.set_ylabel('Domain Type')
ax.set_title('Model Variation by Domain Type\n(SASA)')
ax.invert_yaxis()

# 3. Catalytic vs Linker comparison
ax = axes[1, 0]
catalytic_cv = domain_cv[~domain_cv['is_linker']]['radius_of_gyration_ca'].dropna()
linker_cv = domain_cv[domain_cv['is_linker']]['radius_of_gyration_ca'].dropna()

ax.hist(catalytic_cv, bins=30, alpha=0.7, label=f'Catalytic (n={len(catalytic_cv)}, μ={catalytic_cv.mean():.2f}%)', 
        color='#3498db', density=True)
ax.hist(linker_cv, bins=30, alpha=0.7, label=f'Linker (n={len(linker_cv)}, μ={linker_cv.mean():.2f}%)', 
        color='#e74c3c', density=True)
ax.set_xlabel('CV (%) for Radius of Gyration')
ax.set_ylabel('Density')
ax.set_title('Catalytic vs Linker Domain Variation')
ax.legend()

# 4. CV across all properties comparison
ax = axes[1, 1]
prop_means = [domain_cv[col].mean() for col in numeric_cols]
prop_stds = [domain_cv[col].std() for col in numeric_cols]
ax.barh(numeric_cols, prop_means, xerr=prop_stds, color=plt.cm.plasma(np.linspace(0.2, 0.8, len(numeric_cols))), capsize=3)
ax.set_xlabel('Mean CV (%)')
ax.set_ylabel('Property')
ax.set_title('Which Properties Show Most Model Variation?')
ax.invert_yaxis()

plt.tight_layout()
plt.show()


---
## 4. Statistical Tests for Model Differences


In [ ]:
# ANOVA test - do models differ significantly?
from scipy.stats import f_oneway, kruskal

print("Statistical Tests: Are models significantly different?")
print("="*60)

# For each property, test if models differ
for col in numeric_cols:
    groups = [module_mp[module_mp['model_num'] == m][col].dropna() for m in range(5)]
    groups = [g for g in groups if len(g) > 10]
    
    if len(groups) >= 2:
        # Kruskal-Wallis (non-parametric)
        stat, p = kruskal(*groups)
        print(f"\n{col}:")
        print(f"  Kruskal-Wallis H={stat:.2f}, p={p:.2e}")
        if p < 0.05:
            print(f"  → Models differ significantly (p < 0.05)")
        else:
            print(f"  → No significant difference between models")


In [ ]:
# Calculate ICC (Intraclass Correlation Coefficient) for model agreement
# Higher ICC = better agreement between models

def calculate_icc(data, group_col, value_col):
    """Calculate ICC(2,1) for agreement between raters/models"""
    groups = data.groupby(group_col)[value_col].apply(list)
    groups = groups[groups.apply(len) >= 2]
    
    if len(groups) < 10:
        return np.nan
    
    # Calculate between-group and within-group variance
    all_values = []
    group_means = []
    for g in groups:
        all_values.extend(g)
        group_means.append(np.mean(g))
    
    grand_mean = np.mean(all_values)
    n_groups = len(groups)
    k = np.mean([len(g) for g in groups])
    
    # Between-group sum of squares
    ss_between = sum(len(g) * (np.mean(g) - grand_mean)**2 for g in groups)
    # Within-group sum of squares
    ss_within = sum(sum((x - np.mean(g))**2 for x in g) for g in groups)
    
    ms_between = ss_between / (n_groups - 1) if n_groups > 1 else 0
    ms_within = ss_within / (n_groups * (k - 1)) if k > 1 else 0
    
    # ICC(2,1)
    icc = (ms_between - ms_within) / (ms_between + (k - 1) * ms_within) if (ms_between + (k - 1) * ms_within) > 0 else 0
    return max(0, icc)  # ICC can't be negative in practice

print("\nIntraclass Correlation Coefficients (Model Agreement)")
print("="*60)
print("Higher ICC = More agreement between models (0-1 scale)")
print()

icc_results = {}
for col in numeric_cols:
    icc = calculate_icc(module_mp.dropna(subset=[col]), 'base_structure', col)
    icc_results[col] = icc
    print(f"{col:30s}: ICC = {icc:.4f}")

# Visualize ICC
fig, ax = plt.subplots(figsize=(10, 6))
colors = plt.cm.RdYlGn([v for v in icc_results.values()])
bars = ax.barh(list(icc_results.keys()), list(icc_results.values()), color=colors)
ax.axvline(0.75, color='green', linestyle='--', linewidth=2, label='Good agreement (0.75)')
ax.axvline(0.5, color='orange', linestyle='--', linewidth=2, label='Moderate agreement (0.5)')
ax.set_xlabel('ICC (Intraclass Correlation)')
ax.set_ylabel('Property')
ax.set_title('Model Agreement by Property\n(Higher = More Consistent Predictions)')
ax.set_xlim(0, 1)
ax.legend(loc='lower right')
plt.tight_layout()
plt.show()


---
## 5. Recommendations: Single Model vs Ensemble


In [ ]:
# Final summary and recommendations
print("="*70)
print("SUMMARY: AlphaFold Model Variation Analysis")
print("="*70)

print("\n1. COEFFICIENT OF VARIATION (CV) SUMMARY:")
print("-" * 50)
for col in numeric_cols:
    cv_mean = module_cv[col].mean()
    cv_median = module_cv[col].median()
    print(f"   {col:25s}: Mean CV = {cv_mean:.2f}%, Median CV = {cv_median:.2f}%")

print("\n2. MODEL AGREEMENT (ICC) SUMMARY:")
print("-" * 50)
for col, icc in icc_results.items():
    agreement = "Excellent" if icc > 0.9 else "Good" if icc > 0.75 else "Moderate" if icc > 0.5 else "Poor"
    print(f"   {col:25s}: ICC = {icc:.3f} ({agreement})")

print("\n3. RECOMMENDATIONS:")
print("-" * 50)
avg_cv = np.mean([module_cv[col].mean() for col in numeric_cols])
avg_icc = np.mean(list(icc_results.values()))

if avg_cv < 2.0 and avg_icc > 0.9:
    print("   ✓ Models show EXCELLENT agreement")
    print("   → Using any single model is likely sufficient")
    print("   → Model 0 is typically recommended (highest confidence)")
elif avg_cv < 5.0 and avg_icc > 0.75:
    print("   ✓ Models show GOOD agreement")
    print("   → Single model is acceptable for most analyses")
    print("   → Consider averaging for publication-quality results")
else:
    print("   ⚠ Models show MODERATE variation")
    print("   → Recommend using ensemble average")
    print("   → Report standard deviation across models as uncertainty")

print(f"\n   Overall Average CV: {avg_cv:.2f}%")
print(f"   Overall Average ICC: {avg_icc:.3f}")


In [ ]:
# Create ensemble-averaged dataset (for reference)
print("\nCreating ensemble-averaged module data...")

module_ensemble = module_mp.groupby('base_structure').agg({
    **{col: ['mean', 'std'] for col in numeric_cols},
    'module': 'first',
    'bgc': 'first'
}).reset_index()

# Flatten column names
module_ensemble.columns = ['_'.join(col).strip('_') if isinstance(col, tuple) else col 
                           for col in module_ensemble.columns]

print(f"Ensemble dataset shape: {module_ensemble.shape}")
print("\nExample ensemble data:")
display(module_ensemble.head())


# AlphaFold Model Variation Analysis

This notebook analyzes the **structural variation between different AlphaFold model predictions** (model_0, model_1, model_2, etc.) for PKS modules.

## Key Questions:
1. How consistent are structural predictions across different models?
2. Which structural properties show the most/least variation?
3. Which domains or modules show highest prediction uncertainty?
4. Should we use a single model, average, or ensemble for downstream analysis?

## Data Sources:
- `domain_macroproperties.csv` - Individual domain structures
- `MP_PKS.csv` - Full module structures
- `IDO_out.csv` - Inter-domain spatial relationships


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 100)

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

print("Libraries loaded successfully")


---
## 1. Load and Parse Data

Extract model numbers from filenames to enable comparison across predictions.


In [ ]:
# Load data
domain_mp = pd.read_csv('domain_macroproperties.csv', index_col=0)
module_mp = pd.read_csv('MP_PKS.csv')
ido_df = pd.read_csv('IDO_out.csv', low_memory=False)

print(f"Domain macroproperties: {domain_mp.shape}")
print(f"Module macroproperties: {module_mp.shape}")
print(f"Inter-domain organization: {ido_df.shape}")


In [ ]:
# Extract model number and structure identifier from filenames
def parse_filename(filename):
    """Extract model number and base structure ID from filename"""
    # Pattern: ..._model_X.npz or ..._model_X_...
    model_match = re.search(r'model_(\d+)', filename.lower())
    model_num = int(model_match.group(1)) if model_match else None
    
    # Get base structure ID (everything before model_X)
    if model_match:
        base_id = filename[:model_match.start()].rstrip('_')
    else:
        base_id = filename.replace('.npz', '')
    
    return {'model_num': model_num, 'base_id': base_id}

# Apply to domain data
domain_info = domain_mp['filename'].apply(parse_filename).apply(pd.Series)
domain_mp = pd.concat([domain_mp, domain_info], axis=1)

# Apply to module data
module_info = module_mp['filename'].apply(parse_filename).apply(pd.Series)
module_mp = pd.concat([module_mp, module_info], axis=1)

print("Model number distribution (Modules):")
print(module_mp['model_num'].value_counts().sort_index())

print("\nModel number distribution (Domains):")
print(domain_mp['model_num'].value_counts().sort_index())


---
## 2. Module-Level Model Variation Analysis

Compare structural properties across the 5 AlphaFold models for each module.


In [ ]:
# Group modules by base_id and calculate variation statistics
numeric_cols = ['n_ca_atoms', 'n_heavy_atoms', 'radius_of_gyration_ca', 'sasa', 'ses_area', 'vdw_volume']

# Calculate coefficient of variation (CV) for each structure across its models
module_variation = module_mp.groupby('base_id')[numeric_cols].agg(['mean', 'std', 'count'])

# Calculate CV = std/mean for each property
for col in numeric_cols:
    module_variation[(col, 'cv')] = module_variation[(col, 'std')] / module_variation[(col, 'mean')] * 100

# Flatten column names
module_variation.columns = ['_'.join(col).strip() for col in module_variation.columns.values]

# Filter to structures with multiple models
multi_model = module_variation[module_variation[f'{numeric_cols[0]}_count'] > 1].copy()

print(f"Structures with multiple models: {len(multi_model)}")
print(f"\nCoefficient of Variation (%) across models:")


In [ ]:
# Summary statistics for CV across all structures
cv_cols = [f'{col}_cv' for col in numeric_cols]
cv_summary = multi_model[cv_cols].describe()
cv_summary.columns = [c.replace('_cv', '') for c in cv_cols]
print("CV Statistics (%) - Lower = more consistent across models:")
display(cv_summary.round(4))
